In [2]:
import sqlalchemy
import pandas as pd
import numpy as np

In [2]:
n_lats = (40.92-40.498)/.001 +1
lats = np.linspace(40.498, 40.92, int(n_lats))

n_longs = ((74.254)-73.681)/.001 + 1
longs = np.linspace(-74.254, -73.681, int(n_longs))

In [4]:
from weather import get_dates_list
from datetime import datetime

# get most recent date from NYC's year to date arrests
final_date_df = pd.read_csv('../nyc_ytd.csv', 
                    usecols=['ARREST_DATE'],
                    nrows=2)
final_date = datetime.strptime(final_date_df.loc[0, 'ARREST_DATE'], '%m/%d/%Y')

initial_date = datetime.strptime('01/01/2006', '%m/%d/%Y')

dates = get_dates_list(initial_date, final_date)


In [ ]:
from itertools import product

# lat_long_df = pd.DataFrame(product(lats, longs))
lat_long_arr = np.array(list(product(lats, longs, dates)))

In [ ]:
lat_long_arr.shape